### Align Two Strings Using Affine Gap Penalties

https://rosalind.info/problems/ba5j

In [2]:
import numpy as np

def read_blosum62(file_path):
    with open(file_path) as f:
        lines = [line.strip().split() for line in f if line.strip()]
    headers = lines[0]
    matrix = {}
    for row in lines[1:]:
        aa = row[0]
        scores = list(map(int, row[1:]))
        for h, s in zip(headers, scores):
            matrix[(aa, h)] = s
    return matrix

def affine_gap_alignment(v, w, blosum, sigma=11, epsilon=1):
    n, m = len(v), len(w)
    NEG_INF = float('-inf')

    M = np.full((n+1, m+1), NEG_INF)
    X = np.full((n+1, m+1), NEG_INF)
    Y = np.full((n+1, m+1), NEG_INF)
    
    M[0,0] = 0
    for i in range(1, n+1):
        X[i,0] = -sigma - (i-1)*epsilon
    for j in range(1, m+1):
        Y[0,j] = -sigma - (j-1)*epsilon

    backM = [[None]*(m+1) for _ in range(n+1)]
    backX = [[None]*(m+1) for _ in range(n+1)]
    backY = [[None]*(m+1) for _ in range(n+1)]

    for i in range(1, n+1):
        for j in range(1, m+1):
            s = blosum[(v[i-1], w[j-1])]
            # M matrix
            choices = [M[i-1,j-1], X[i-1,j-1], Y[i-1,j-1]]
            M[i,j] = max(choices) + s
            backM[i][j] = ['M','X','Y'][np.argmax(choices)]
            # X matrix (gap in w)
            choices = [M[i-1,j] - sigma, X[i-1,j] - epsilon]
            X[i,j] = max(choices)
            backX[i][j] = ['M','X'][np.argmax(choices)]
            # Y matrix (gap in v)
            choices = [M[i,j-1] - sigma, Y[i,j-1] - epsilon]
            Y[i,j] = max(choices)
            backY[i][j] = ['M','Y'][np.argmax(choices)]

    # find max
    matrices = [M[n,m], X[n,m], Y[n,m]]
    max_score = max(matrices)
    matrix_name = ['M','X','Y'][np.argmax(matrices)]

    # backtrack
    i, j = n, m
    v_aln, w_aln = [], []
    mat = matrix_name

    while i > 0 or j > 0:
        if mat == 'M':
            prev = backM[i][j]
            v_aln.append(v[i-1])
            w_aln.append(w[j-1])
            i -= 1
            j -= 1
        elif mat == 'X':
            prev = backX[i][j]
            v_aln.append(v[i-1])
            w_aln.append('-')
            i -= 1
        elif mat == 'Y':
            prev = backY[i][j]
            v_aln.append('-')
            w_aln.append(w[j-1])
            j -= 1
        mat = prev

    return int(max_score), ''.join(reversed(v_aln)), ''.join(reversed(w_aln))

if __name__ == "__main__":
    with open("data/rosalind_ba5j.txt") as f:
        v = f.readline().strip()
        w = f.readline().strip()

    blosum = read_blosum62("data/BLOSUM62.txt")
    score, v_aln, w_aln = affine_gap_alignment(v, w, blosum)
    print(score)
    print(v_aln)
    print(w_aln)


97
NHRGWDRRLNLPPN--ESANG--GHSEEWRPMMEPIGTAHFKHVPGFTVPFEPMMHEEMKWPTDDKQWAPLI-----GDFYKMH
N-------LNLPPNWRESTHKICQHSEEYRPM--------FGHVPGFTVPFE--------------QWAPLIHSAPVWDKQGMH


### Find a Middle Edge in an Alignment Graph in Linear Space

https://rosalind.info/problems/ba5k

错误的尝试：

In [7]:
import numpy as np

def read_blosum62(file_path):
    with open(file_path) as f:
        lines = [line.strip().split() for line in f if line.strip()]
    headers = lines[0]
    blosum = {}
    for row in lines[1:]:
        aa = row[0]
        scores = list(map(int, row[1:]))
        for h, s in zip(headers, scores):
            blosum[(aa, h)] = s
    return blosum

def middle_edge(v, w, blosum, sigma=5):
    n, m = len(v), len(w)
    mid = m // 2

    def forward(v, w):
        prev = np.arange(len(v)+1) * -sigma
        for j in range(1, len(w)+1):
            curr = np.zeros(len(v)+1)
            curr[0] = -sigma * j
            for i in range(1, len(v)+1):
                match = prev[i-1] + blosum[(v[i-1], w[j-1])]
                delete = prev[i] - sigma
                insert = curr[i-1] - sigma
                curr[i] = max(match, delete, insert)
            prev = curr
        return prev

    # Forward scores up to middle column
    from_source = forward(v, w[:mid])
    # Backward scores from middle+1 to end (reverse both)
    to_sink = forward(v[::-1], w[mid:][::-1])[::-1]

    # Best i index at middle column
    scores = from_source + to_sink
    i_star = np.argmax(scores)
    j_star = mid

    # Compute next step (to determine edge direction)
    next_scores = {
        (i_star+1, j_star): from_source[i_star] - sigma + to_sink[i_star+1] if i_star < n else -np.inf,  # deletion
        (i_star, j_star+1): from_source[i_star] - sigma + to_sink[i_star] if j_star < m else -np.inf,    # insertion
        (i_star+1, j_star+1): from_source[i_star] + blosum.get((v[i_star], w[j_star]), -np.inf) + to_sink[i_star+1]
        if i_star < n and j_star < m else -np.inf,  # match/mismatch
    }

    (k, l) = max(next_scores, key=next_scores.get)
    return (i_star, j_star), (k, l)

if __name__ == "__main__":
    with open("data/rosalind_ba5k.txt") as f:
        v = f.readline().strip()
        w = f.readline().strip()
    blosum = read_blosum62("data/BLOSUM62.txt")
    edge = middle_edge(v, w, blosum)
    print(f"({edge[0][0]}, {edge[0][1]}) ({edge[1][0]}, {edge[1][1]})")


(532, 522) (532, 523)


在计算下一步边时，要确保反向得分索引与原序列对齐：

In [22]:
import numpy as np

def read_blosum62(file_path):
    """Read BLOSUM62 scoring matrix into a dictionary."""
    with open(file_path) as f:
        lines = [line.strip().split() for line in f if line.strip()]
    headers = lines[0]
    blosum = {}
    for row in lines[1:]:
        aa = row[0]
        scores = list(map(int, row[1:]))
        for h, s in zip(headers, scores):
            blosum[(aa, h)] = s
    return blosum

def middle_edge(v, w, blosum, sigma=5):
    n, m = len(v), len(w)
    mid = m // 2

    def forward_score(v, w):
        """Compute DP column scores in linear space."""
        prev = np.arange(len(v)+1) * -sigma
        for j in range(1, len(w)+1):
            curr = np.zeros(len(v)+1)
            curr[0] = -sigma * j
            for i in range(1, len(v)+1):
                match = prev[i-1] + blosum[(v[i-1], w[j-1])]
                delete = prev[i] - sigma
                insert = curr[i-1] - sigma
                curr[i] = max(match, delete, insert)
            prev = curr
        return prev

    # Forward scores up to middle column
    from_source = forward_score(v, w[:mid])
    # Backward scores from middle+1 to end (reverse both)
    to_sink = forward_score(v[::-1], w[mid:][::-1])[::-1]

    # Find best i at middle column
    scores = from_source + to_sink
    i_star = np.argmax(scores)
    j_star = mid

    # Determine next edge direction
    diag = from_source[i_star] + (blosum[(v[i_star], w[j_star])] if i_star < n and j_star < m else -np.inf) + (to_sink[i_star+1] if i_star < n else -np.inf)
    down = from_source[i_star] - sigma + (to_sink[i_star+1] if i_star < n else -np.inf)
    right = from_source[i_star] - sigma + (to_sink[i_star] if j_star < m else -np.inf)

    next_scores = {
        (i_star+1, j_star+1): diag,
        (i_star+1, j_star): down,
        (i_star, j_star+1): right
    }
    (k, l) = max(next_scores, key=next_scores.get)

    return (i_star, j_star), (k, l)

if __name__ == "__main__":
    # Read input sequences
    with open("data/rosalind_ba5k.txt") as f:
        v = f.readline().strip()
        w = f.readline().strip()

    blosum = read_blosum62("data/BLOSUM62.txt")
    edge = middle_edge(v, w, blosum)
    print(f"({edge[0][0]}, {edge[0][1]}) ({edge[1][0]}, {edge[1][1]})")


(532, 522) (533, 523)


### Align Two Strings Using Linear Space

https://rosalind.info/problems/ba5l

In [14]:
import numpy as np

# Load BLOSUM62
def load_blosum62(path="data/BLOSUM62.txt"):
    with open(path) as f:
        lines = [line.strip() for line in f.readlines() if line.strip() and not line.startswith('#')]
    headers = lines[0].split()
    blosum62 = {}
    for line in lines[1:]:
        parts = line.split()
        row_aa = parts[0]
        scores = list(map(int, parts[1:]))
        blosum62[row_aa] = dict(zip(headers, scores))
    return blosum62

BLOSUM62 = load_blosum62()
GAP_PENALTY = 5

# Linear-space score for a substring
def linear_space_score(v, w):
    n = len(v)
    m = len(w)
    prev = [-j*GAP_PENALTY for j in range(m+1)]
    for i in range(1, n+1):
        curr = [0]*(m+1)
        curr[0] = -i*GAP_PENALTY
        for j in range(1, m+1):
            match = prev[j-1] + BLOSUM62[v[i-1]][w[j-1]]
            delete = prev[j] - GAP_PENALTY
            insert = curr[j-1] - GAP_PENALTY
            curr[j] = max(match, delete, insert)
        prev = curr
    return prev

# Hirschberg's algorithm
def hirschberg(v, w):
    if len(v) == 0:
        return "-"*len(w), w
    elif len(w) == 0:
        return v, "-"*len(v)
    elif len(v) == 1 or len(w) == 1:
        return needleman_wunsch(v, w)
    else:
        i = len(v)//2
        score_l = linear_space_score(v[:i], w)
        score_r = linear_space_score(v[i:][::-1], w[::-1])
        total = [l+r for l, r in zip(score_l, score_r[::-1])]
        k = total.index(max(total))
        v1, w1 = hirschberg(v[:i], w[:k])
        v2, w2 = hirschberg(v[i:], w[k:])
        return v1+v2, w1+w2

# Needleman-Wunsch for small strings
def needleman_wunsch(v, w):
    n = len(v)
    m = len(w)
    dp = np.zeros((n+1, m+1), dtype=int)
    for i in range(n+1):
        dp[i][0] = -i*GAP_PENALTY
    for j in range(m+1):
        dp[0][j] = -j*GAP_PENALTY
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = dp[i-1][j-1] + BLOSUM62[v[i-1]][w[j-1]]
            delete = dp[i-1][j] - GAP_PENALTY
            insert = dp[i][j-1] - GAP_PENALTY
            dp[i][j] = max(match, delete, insert)
    # Traceback
    aligned_v, aligned_w = "", ""
    i, j = n, m
    while i > 0 and j > 0:
        score = dp[i][j]
        if score == dp[i-1][j-1] + BLOSUM62[v[i-1]][w[j-1]]:
            aligned_v = v[i-1] + aligned_v
            aligned_w = w[j-1] + aligned_w
            i -= 1
            j -= 1
        elif score == dp[i-1][j] - GAP_PENALTY:
            aligned_v = v[i-1] + aligned_v
            aligned_w = "-" + aligned_w
            i -= 1
        else:
            aligned_v = "-" + aligned_v
            aligned_w = w[j-1] + aligned_w
            j -= 1
    while i > 0:
        aligned_v = v[i-1] + aligned_v
        aligned_w = "-" + aligned_w
        i -= 1
    while j > 0:
        aligned_v = "-" + aligned_v
        aligned_w = w[j-1] + aligned_w
        j -= 1
    return aligned_v, aligned_w

# Compute alignment score
def alignment_score(aligned_v, aligned_w):
    score = 0
    for a, b in zip(aligned_v, aligned_w):
        if a == "-" or b == "-":
            score -= GAP_PENALTY
        else:
            score += BLOSUM62[a][b]
    return score

# Read dataset
with open("data/rosalind_ba5l.txt") as f:
    sequences = [line.strip() for line in f if line.strip()]
v, w = sequences[0], sequences[1]

aligned_v, aligned_w = hirschberg(v, w)
score = alignment_score(aligned_v, aligned_w)

print(score)
print(aligned_v)
print(aligned_w)


17377
ASMNSRYLHCMWSMSFL-TEV--Q-GHEWVCAKGYYY-----GKVK--VMHCMVAEVRPQYD-M--MEVRADAMYHFNKAMDNKLCFMQDSLVMPERMAGPDAY-C-MNDDCSSHPFCECSAKY---KFRPFTHWRDGHACVHKWN-HAVPYGWDCFRLDAWKNCKPK-SEYKHKGLPTMPACMIAETHFIAMMDPYKPHFDMCFIKKDTAQGMFAICPYPLWLKWAKFSDNSIDTTKVYLNMDSWQHGHDHWCPENPERKNRIRRHIPPPHVCWVKTPKKHSTCCVKNFD-AW-WIE-HIPVNHNTEMANTNSMPLWYAEPCNSDMYALTVAAIRTMNGQRGNCC-DPFMPQQKIVCGAVC-P-----A--TWWFWFYVVYV-VEEVVHKYNH---YI-MCGGPGVRRAMRFKL-DC-IMG-SKAKMGHIVHLTRWELTFAKMNYITVQKCQHNWY-W-CA--LQVSSQEFEVK-PY-R--SNETVGQTYHHWALPVWPGRGKTLVYEFHANYTSEFAQK-----TYHLGRMRLVTDPRWWCLFASIEQWNQEVYVWHI-QQEPPMGNMEDNE-LKERRIHMIFVNELFDHRNIRKALSERYHM-RQHLSAAEKSWNDFCIDHICNMQSLWVQAITAIQQCIITETFQDRAPERGGSEIFDMCHIDEQNSEWVIMYEMQANEWMSDFQTKFITCIGHQCQTVREGSMEEKQHTMKRACH--YIRYLNHGQYGYYRHGVKWVEFELMWW-H-G-SCIWGSGPRCHCRYVHLNSDISRCTYEWDPENGNFM-I---C--N-WIKPKYCHCLFEWWHNYIEAILHGCQYTYKPFPLRMWMDNEEDACAEQWVRQAKDW-KMR-LKCYQGTTWAG-EAAWMPCVPFFM-R-----FLDCVHCCLDPFP----N--WHFQSNLYFAIMDIAIGEVHRRNF-CEDAGLSDA-H--CP-PIWDRWLGLDAGEHFYTVAC-WMDVNIWRSWANCAHGFTEH

### Find a Highest-Scoring Multiple Sequence Alignment

https://rosalind.info/problems/ba5m

In [11]:
def multiple_lcs(s1, s2, s3):
    n, m, l = len(s1), len(s2), len(s3)
    
    # DP table
    dp = [[[0]*(l+1) for _ in range(m+1)] for _ in range(n+1)]
    
    # Fill DP table
    for i in range(1, n+1):
        for j in range(1, m+1):
            for k in range(1, l+1):
                if s1[i-1] == s2[j-1] == s3[k-1]:
                    dp[i][j][k] = dp[i-1][j-1][k-1] + 1
                else:
                    dp[i][j][k] = max(dp[i-1][j][k], dp[i][j-1][k], dp[i][j][k-1])
    
    # Reconstruct alignment
    i, j, k = n, m, l
    a1, a2, a3 = [], [], []
    
    while i > 0 or j > 0 or k > 0:
        if i > 0 and j > 0 and k > 0 and s1[i-1] == s2[j-1] == s3[k-1]:
            a1.append(s1[i-1])
            a2.append(s2[j-1])
            a3.append(s3[k-1])
            i -= 1; j -= 1; k -= 1
        elif i > 0 and dp[i][j][k] == dp[i-1][j][k]:
            a1.append(s1[i-1]); a2.append('-'); a3.append('-')
            i -= 1
        elif j > 0 and dp[i][j][k] == dp[i][j-1][k]:
            a1.append('-'); a2.append(s2[j-1]); a3.append('-')
            j -= 1
        else:
            a1.append('-'); a2.append('-'); a3.append(s3[k-1])
            k -= 1
    
    return dp[n][m][l], ''.join(reversed(a1)), ''.join(reversed(a2)), ''.join(reversed(a3))


# Read dataset from file
with open("data/rosalind_ba5m.txt") as f:
    lines = [line.strip() for line in f if line.strip()]

s1, s2, s3 = lines[:3]  # Assuming exactly 3 strings in the file

score, align1, align2, align3 = multiple_lcs(s1, s2, s3)
print(score)
print(align1)
print(align2)
print(align3)


12
-------TC--CCTAAT------T----TAAC-CCACTGA--C---TG--C-A----AAGCC
-----TA-CGA---AAT--GCGCT---G---CG-----GA--C-GA-GACCTA-CTG-----
GGGGG---C-----AATGA----TGGG----C------GATGCT---G--C-AT--------
